In [3]:
#读取数据
import pandas as pd
data_path = 'data.csv'
data= pd.read_csv(data_path)[1:]

In [4]:
data

,continue_drop,student_id,gender,caste,mathematics_marks,english_marks,science_marks,science_teacher,languages_teacher,guardian,internet,school_id,total_students,total_toilets,establishment_year
1,continue,s00002,F,BC,0.290,0.512,0.290,4,7,mother,True,328,356,14.0,1943.0
2,continue,s00003,F,OC,0.602,0.666,0.602,4,2,mother,False,322,179,8.0,1955.0
3,continue,s00004,F,BC,0.378,0.526,0.378,8,7,mother,True,305,354,86.0,1986.0
4,continue,s00005,F,OC,0.536,0.614,0.536,9,4,other,True,360,273,2.0,1995.0
5,continue,s00006,F,BC,0.594,0.519,0.594,4,8,mother,True,333,335,43.0,1916.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095,continue,s19096,M,OC,0.323,0.429,0.323,2,7,mother,True,320,384,28.0,1909.0
19096,continue,s19097,M,BC,0.633,0.885,0.633,6,2,mother,True,378,121,28.0,1971.0
19097,continue,s19098,M,SC,0.337,0.283,0.337,2,6,other,True,378,121,28.0,1971.0
19098,continue,s19099,M,BC,0.344,0.351,0.344,6,0,other,True,374,207,28.0,1896.0


In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])
data

,continue_drop,student_id,gender,caste,mathematics_marks,english_marks,science_marks,science_teacher,languages_teacher,guardian,internet,school_id,total_students,total_toilets,establishment_year
1,0,0,0,0,48,110,48,4,7,2,1,28,77,6,42
2,0,1,0,1,187,203,187,4,2,2,0,22,46,4,48
3,0,2,0,0,86,122,86,8,7,2,1,5,76,21,65
4,0,3,0,1,157,174,157,9,4,3,1,60,65,1,68
5,0,4,0,0,185,115,185,4,8,2,1,33,73,14,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095,0,19094,1,1,58,64,58,2,7,2,1,20,79,11,25
19096,0,19095,1,0,200,276,200,6,2,2,1,78,28,11,57
19097,0,19096,1,2,63,18,63,2,6,3,1,78,28,11,57
19098,0,19097,1,0,67,36,67,6,0,3,1,74,54,11,17


In [6]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader,TensorDataset
from tqdm.autonotebook import tqdm
from torch.utils.tensorboard import SummaryWriter
import pdb

In [21]:
class LinearLayer(nn.Module):
    def __init__(self, in_f, out_f, activate=2):
        super().__init__()
        a = [torch.nn.Sigmoid(),torch.nn.Tanh(),torch.nn.LeakyReLU()]
        net = [
            nn.Linear(in_f, out_f),
            a[activate],
            nn.BatchNorm1d(out_f)
        ]
        self.net = nn.Sequential(*net)
    def forward(self, x):
        x = self.net(x)
        return x

# def MLP_model(num_layers, hidden_dim,activate):
#     a = [torch.nn.Sigmoid(),torch.nn.Tanh(),torch.nn.LeakyReLU()]
#     layers = []
#     for i in range(num_layers):
#         layers.append(LinearLayer(hidden_dim[i], hidden_dim[i+1]))
#         layers.append(a[activate])
#         layers.append(nn.BatchNorm1d(hidden_dim[i+1]))
#         if i < num_layers - 1:
#             layers.append(nn.Dropout(0.2))
#     #layers.append(nn.Softmax(dim=1))
#     return nn.Sequential(*layers)

class MLP_model(nn.Module):
    def __init__(self, nomask_dims, mask_dims, share_dims, activate=2):
        super().__init__()
        a = [torch.nn.Sigmoid(),torch.nn.Tanh(),torch.nn.LeakyReLU()]
        
        nomask_layers = []
        for i in range(len(nomask_dims) - 1):
            nomask_layers.append(LinearLayer(nomask_dims[i], nomask_dims[i + 1], activate))
        self.nomask_layers = nn.Sequential(*nomask_layers)
        
        mask_layers = []
        for i in range(len(mask_dims) - 1):
            mask_layers.append(LinearLayer(mask_dims[i], mask_dims[i + 1], activate))
        self.mask_layers = nn.Sequential(*mask_layers)
        
        share_layers = []
        share_layers.append(LinearLayer(mask_dims[-1] + nomask_dims[-1], share_dims[0], activate))
        for i in range(len(share_dims) - 1):
            share_layers.append(LinearLayer(share_dims[i], share_dims[i + 1], activate))
        self.share_layers = nn.Sequential(*share_layers)
    
    def forward(self, nomask_input, mask_input):
        x = self.nomask_layers(nomask_input)
        mask = torch.randint(2, mask_input.shape).to(mask_input.device)
        y = self.mask_layers(mask_input * mask)
        mid = torch.cat((x, y), dim=-1)
        out = self.share_layers(mid)
        return out
        

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

In [24]:
def read_data():
    data=pd.read_csv(data_path)
    labelencoder = LabelEncoder()
    for col in data.columns:
        data[col] = labelencoder.fit_transform(data[col])
    data.head()
    np.save("data", data[1:])


def get_dataset(data):
    data_class = [[], []]
    for i in range(data.shape[0]):
        data_class[data[i][0].long()].append(data[i])
    
    train_dataset = [[], []]
    test_dataset = []
    for i in range(len(data_class[0])):
        if i < len(data_class[0]) * 0.8:
            train_dataset[0].append(data_class[0][i])
        else:
            test_dataset.append(data_class[0][i])
            
    for i in range(len(data_class[1])):
        if i < len(data_class[1]) * 0.8:
            train_dataset[1].append(data_class[1][i])
        else:
            test_dataset.append(data_class[1][i])
    train_dataset_con = TensorDataset(torch.cat(train_dataset[0]).reshape(-1, 15))
    train_dataset_drop = TensorDataset(torch.cat(train_dataset[1]).reshape(-1, 15))
    train_dataset_all = TensorDataset(torch.cat(train_dataset[0] + train_dataset[1]).reshape(-1, 15))
    test_dataset_all = TensorDataset(torch.cat(test_dataset).reshape(-1, 15))
    return train_dataset_con, train_dataset_drop, train_dataset_all, test_dataset_all


def train():
    writer = SummaryWriter('log')
    model.train()
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = torch.nn.CrossEntropyLoss()    
    
    train_loader_con = DataLoader(train_dataset_con, batch_size=96, shuffle=True)
    train_loader_drop = iter(DataLoader(train_dataset_drop, batch_size=32, shuffle=True))
    
    size = len(train_dataset_con) + len(train_dataset_drop)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=2000,
                                                               verbose=True, threshold=0.002, threshold_mode='rel',
                                                               cooldown=2000)
    
    step = 0
    pbar = tqdm(range(epoch), initial=0, dynamic_ncols=True, smoothing=0.01)
    for i in pbar:
        # break
        for j, batch_con in enumerate(train_loader_con):
            try:
                batch_drop = next(train_loader_drop)
            except:
                del train_loader_drop
                train_loader_drop = iter(DataLoader(train_dataset_drop, batch_size=32, shuffle=True)) 
                batch_drop = next(train_loader_drop)
            
            batch = torch.cat(batch_drop, dim=0)
            # pdb.set_trace()
            batch = torch.cat((batch, batch_con[0]), dim=0)
            
            optim.zero_grad()
            # input = batch[:, 2:15].float()
            nomask_input = batch[:, [2,3,7,8,9,10,11,12,13,14]].float().to(device)
            mask_input = batch[:, 4:7].float().to(device)
            ground_truth = batch[:,0].long().to(device)
            out_put = model(nomask_input, mask_input)
            loss = loss_fn(out_put, ground_truth)
            loss.backward()
            optim.step()
            scheduler.step(loss)
            
            writer.add_scalar('loss', loss, step)
            
            pbar.set_description(
                        (
                            f'iter: {step} loss: {loss:.4f}'
                        )
                    )
            step += 1
    torch.save(model, f"model/model.pth")

def test():
    model=torch.load("model/model.pth").to(torch.device('cuda:0'))
    model.eval()
    
    print("训练集输出结果:")
    a = np.zeros(shape=(2, 2))
    dataloader = DataLoader(train_dataset_all, batch_size=1, shuffle=True)
    for i, batch in enumerate(dataloader):
        nomask_input = batch[0][:, [2,3,7,8,9,10,11,12,13,14]].float().to(device)
        mask_input = batch[0][:, 4:7].float().to(device)
        out_put = model(nomask_input, mask_input)
        a[int(batch[0][0][0]), torch.argmax(out_put, dim=1)] += 1
    print(a)
    print(f'准确率{(a[0][0] + a[1][1]) / len(train_dataset_all)}')
    
    print("测试集输出结果:")
    a = np.zeros(shape=(2, 2))
    dataloader = DataLoader(test_dataset_all, batch_size=1, shuffle=True)
    for i, batch in enumerate(dataloader):
        nomask_input = batch[0][:, [2,3,7,8,9,10,11,12,13,14]].float().to(device)
        mask_input = batch[0][:, 4:7].float().to(device)
        out_put = model(nomask_input, mask_input)
        a[int(batch[0][0][0]), torch.argmax(out_put, dim=1)] += 1
    print(a)
    print(f'准确率{(a[0][0] + a[1][1]) / len(test_dataset_all)}')
    
if __name__ == "__main__":
    read_data()
    epoch = 50 # setting epoch of train
    nomask_dims = [10, 64, 128, 256]
    mask_dims = [3, 64, 128]
    share_dims = [512, 256, 128, 64, 2]
    activate = 2  # setting the activate function of net
    learning_rate = 0.01  # setting learning_rate
    
    data = torch.tensor(np.load('data.npy', allow_pickle=True).astype(np.float)).to(torch.device('cuda:0'))
    train_dataset_con, train_dataset_drop, train_dataset_all, test_dataset_all = get_dataset(data) 
    
    device = torch.device('cuda:0')
    model = MLP_model(nomask_dims, mask_dims, share_dims, activate).to(device)
    model.apply(init_weights)
    # model=torch.load("model/model.pth").to(torch.device('cuda:0'))
    # train()
    test()


/tmp/ipykernel_1520073/1168847178.py:122: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = torch.tensor(np.load('data.npy', allow_pickle=True).astype(np.float)).to(torch.device('cuda:0'))


训练集输出结果:
[[1.4497e+04 6.3000e+01]
 [1.0000e+00 7.1900e+02]]
准确率0.9958115183246073
测试集输出结果:
[[3623.   16.]
 [   0.  180.]]
准确率0.9958104215763289
